In [ ]:
#y_ This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import librosa
from sklearn.model_selection import train_test_split
import datetime as dt
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

X_data = []
y_data = []

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

In [ ]:
label = -2
for dirname, _, filenames in os.walk('/kaggle/input/audio-emotions/Emotions/'):
    label +=1
    print(f"Emotion:{dirname}, Class:{label}")
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        y, sr = librosa.load(os.path.join(dirname, filename))
        mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T,axis=0)
        X_data.append(mfccs)
        y_data.append(int(label)) 

print(X_data[:5])
print(y_data[:5])

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
np.save('X_data.npy', X_data)
np.save('y_data.npy', y_data)
!ls -1

In [ ]:
X_data = load('X_data.npy')
y_data = load('y_data.npy')

In [ ]:
NUM_OF_CLASSES = 8
NUM_OF_ACTORS = 24

In [ ]:
ravdess_speech_data_array = np.asarray(X_data)
y_data_array = np.array(y_data)
print(y_data_array.shape)
labels_categorical = tf.keras.utils.to_categorical(y_data_array)

In [ ]:
import tensorflow as tf

inputs = tf.keras.Input(shape=(1, 40))

x = tf.keras.layers.Dense(64, activation='relu') (inputs)
x = tf.keras.layers.Dense(128, activation='relu') (x)
x = tf.keras.layers.Dropout(rate = 0.2)(x)
x =tf.keras.layers.LSTM(128, return_sequences=True)(x)
x =tf.keras.layers.LSTM(128, return_sequences=True)(x)
x = tf.keras.layers.Dense(128, activation='relu') (x)
x = tf.keras.layers.Dense(128, activation='relu') (x)
x = tf.keras.layers.Dropout(rate = 0.2)(x)

outputs = tf.keras.layers.Dense(7, activation='softmax')(x)

res_model = tf.keras.Model(inputs, outputs)

res_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

print(res_model.summary())

In [ ]:
x_train,x_test,y_train,y_test= train_test_split(np.array(ravdess_speech_data_array),labels_categorical, test_size=0.20, random_state=9, shuffle=True)
x_train,x_val,y_train,y_val= train_test_split(x_train, y_train, test_size=0.20, random_state=9, shuffle=True)
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  tf.keras.callbacks.TensorBoard(log_dir='./log/{}'.format(dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")), write_images=True),
]

# # df_train.drop(["class"], axis=1)
history = res_model.fit(np.expand_dims(x_train, 1), np.expand_dims(y_train, 1), epochs=256, batch_size=256,
          validation_data=(np.expand_dims(x_val, 1), np.expand_dims(y_val, 1)), callbacks=callbacks)

In [ ]:
print("Evaluate on test data")
results = res_model.evaluate(np.expand_dims(x_test, 1), y_test, batch_size=256)
print("test loss, test acc:", results)

In [ ]:
import matplotlib.pyplot as plt
# import Image

print(history.history.keys())
#  "Accuracy"
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig("accuracy.jpg")

plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig("loss.jpg")

plt.show()
